# ALGORITMOS DE BoW

In [1]:
# Leer el archivo con textFile
arquivo = os.path.join('Data', 'text_500lines.txt')
twittsRDD = (sc.textFile(arquivo, 4))

## - Bag of Words

In [2]:
import re
def bagOfWords(array):
    """Elimina la puntuación, cambia a minúsculas y elimina los espacios iniciales y finales, 
        y solo guarda palabras de más de 2 letras.
    Args:
        array: arreglo de strings.

    Returns:
        array: arreglo de strings limpios y preprocesados.
    """
    new = []
    for i in array:
        norm = re.sub(r'[^A-Za-z0-9 ]', '', i).strip().lower()
        if (len(norm) > 2):
            new.append(norm)
    return new

In [3]:
# Separar cada string (línea del texto) en conjunto de palabras
twittsPalavrasRDD = twittsRDD.map(lambda x: x.split())
twittsPalavrasRDD = twittsPalavrasRDD.map(lambda x: bagOfWords(x))
# Mostrar el RDD del texto creado 
print(twittsPalavrasRDD.collect())

[['protester', 'stage', 'diein', 'protest', 'store', 'nyc', 'anger', 'misplace', 'retweet', 'agree'], ['apple', 'intraday', 'comment', 'update', 'range', 'premium', 'user', 'aapl', 'stockaction', 'trading', 'stock', 'mkt'], ['ios', 'app', '2014', 'name', 'elevate', 'brain', 'training', 'iphone', 'app'], ['shit'], ['founder', 'attack', 'boss', 'ridiculous', 'comment'], ['evidence', 'factcheckthis'], ['educate'], ['hard', 'reach', 'buy', 'suicide', 'squad', 'comicstrip', 'apple', 'store', 'turkey'], ['delete', 'music', 'customer', 'ipod'], ['apple', 'intraday', 'comment', 'update', 'range', 'premium', 'user', 'aapl', 'stockaction', 'trading', 'stock', 'mkt'], ['studio', '45000', 'outlet', 'computer', 'need', 'battery', 'future'], ['apple', 'great', 'business', 'aapl', 'investwall', 'aapl'], ['dear', 'love', 'iphone', 'plus', 'lot', 'great', 'achievement', 'iphone', 'reboot', 'day', 'fail'], ['survey', 'feedback', 'iwatch', 'positioning', 'fashion', 'accessory', 'geek', 'centric', 'gadget

## - TF (Term Frecuency)

In [4]:
# Librería para usar funciones matemáticas
import math

# Función TF
def tf(lista):
    """Calculates el TF de cada palabra de la lista.

    Args:
        lista (array): Arreglo de palabras (strings).

    Returns:
        tuplas (array of tuples): Arreglo de tuplas (palabra,tf de la palabra).
    """
    tuplas = [] 
    cantidad = len(lista) # Obtiene la cantidad de elementos de la lista
    # Recorre cada elemento o palabra de la lista y halla su tf
    for i in lista:
        if(i not in [j[0] for j in tuplas]):
            tuplas.append((i,1 + math.log10(lista.count(i)/cantidad))) # TF de una palabra
    return tuplas

In [5]:
# Halla el TF de todas las palabras de cada línea de todo el corpus
tfTwitts = twittsPalavrasRDD.map(lambda x: tf(x))

print(tfTwitts.collect())

[[('protester', 0.0), ('stage', 0.0), ('diein', 0.0), ('protest', 0.0), ('store', 0.0), ('nyc', 0.0), ('anger', 0.0), ('misplace', 0.0), ('retweet', 0.0), ('agree', 0.0)], [('apple', -0.07918124604762489), ('intraday', -0.07918124604762489), ('comment', -0.07918124604762489), ('update', -0.07918124604762489), ('range', -0.07918124604762489), ('premium', -0.07918124604762489), ('user', -0.07918124604762489), ('aapl', -0.07918124604762489), ('stockaction', -0.07918124604762489), ('trading', -0.07918124604762489), ('stock', -0.07918124604762489), ('mkt', -0.07918124604762489)], [('ios', 0.04575749056067513), ('app', 0.34678748622465627), ('2014', 0.04575749056067513), ('name', 0.04575749056067513), ('elevate', 0.04575749056067513), ('brain', 0.04575749056067513), ('training', 0.04575749056067513), ('iphone', 0.04575749056067513)], [('shit', 1.0)], [('founder', 0.30102999566398125), ('attack', 0.30102999566398125), ('boss', 0.30102999566398125), ('ridiculous', 0.30102999566398125), ('comme

## - IDF (Inverse Document Frequency)

In [6]:
# Halla el número de líneas del texto
numDocs = tfTwitts.count()

print(numDocs)

500


In [7]:
# Halla el IDF de cada palabra de todo el texto
idfTwitts = tfTwitts.flatMap(lambda x: [(i[0],1) for i in x]).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0],math.log10(1 + (numDocs/x[1])))).collect()

print(idfTwitts)

[('stage', 1.375247822180056), ('store', 1.150998679049579), ('nyc', 1.3222192947339193), ('aapl', 0.6724399567969696), ('stockaction', 1.752475272897434), ('2014', 1.5961740128049795), ('name', 1.8599099193190791), ('boss', 2.699837725867246), ('educate', 2.699837725867246), ('customer', 2.0043213737826426), ('business', 2.2244467303362647), ('investwall', 2.699837725867246), ('reboot', 2.699837725867246), ('feedback', 2.699837725867246), ('iwatch', 2.399673721481038), ('accessory', 2.699837725867246), ('onlinefootprint', 2.699837725867246), ('turn', 2.100370545117563), ('asus', 2.699837725867246), ('freezing', 2.399673721481038), ('laggy', 2.699837725867246), ('excellence', 2.699837725867246), ('tell', 1.8027737252919758), ('owner', 2.699837725867246), ('facetime', 2.399673721481038), ('seamlessly', 2.699837725867246), ('inc', 1.414973347970818), ('airport', 2.699837725867246), ('board', 2.399673721481038), ('security', 2.100370545117563), ('card', 2.699837725867246), ('clone', 2.699

## - TF-IDF (Term Frequency - Inverse Document Frequency)

In [8]:
def buscar(value,lista):
    """Función buscar un elemento en una lista de tuplas.

    Args:
        lista : Lista de tuplas.
        value (string): Palabra para buscar en una lista de tuplas.
    Returns:
        val (float): Devuelve un valor de una tupla buscada.
    """
    val = 0
    for i in lista:
        if (value == i[0]):
            val = i[1]
    return val

In [9]:
# Halla el td-idf de una palabra multiplicando el valor de su tf y su idf de dicha palabra
tfidf = tfTwitts.map(lambda x: [(i[0], i[1]*float(buscar(i[0],idfTwitts))) for i in x])

print(tfidf.collect())

[[('protester', 0.0), ('stage', 0.0), ('diein', 0.0), ('protest', 0.0), ('store', 0.0), ('nyc', 0.0), ('anger', 0.0), ('misplace', 0.0), ('retweet', 0.0), ('agree', 0.0)], [('apple', -0.05265090382878352), ('intraday', -0.12907245561618694), ('comment', -0.1387631757756703), ('update', -0.11203935281650888), ('range', -0.16630995692413775), ('premium', -0.14274586991053725), ('user', -0.13199737125838618), ('aapl', -0.05324463367139509), ('stockaction', -0.1387631757756703), ('trading', -0.13199737125838618), ('stock', -0.10889383618449928), ('mkt', -0.12907245561618694)], [('ios', 0.08249040171808072), ('app', 0.5143002759613143), ('2014', 0.0730369173241188), ('name', 0.0851048105769488), ('elevate', 0.10178510026608628), ('brain', 0.10980304765936875), ('training', 0.10980304765936875), ('iphone', 0.044989048661627375)], [('shit', 1.8599099193190791)], [('founder', 0.81273213891127), ('attack', 0.7223737699724067), ('boss', 0.81273213891127), ('ridiculous', 0.81273213891127), ('comm

## - N-grams y getNgrams

**Generar 'ngrams' a partir de una secuencia de tokens**

In [10]:
def ngrams(string, n):
    """Función en el que dado un string y un número n, se convierte en un arreglo de palabras,
        y luego se recupera una lista de n-gramas.

    Args:
        string : Una línea del texto.
        n (int): Valor entero para los gramas.
    Returns:
        val (float): Arreglo de ngramas generado.
    """
    listaPalabras = string.split()
    ngramas = []
    for i in range(len(listaPalabras)-(n-1)):
        ngramas.append(' '.join(listaPalabras[i:i+n]))
    return ngramas

**Generar 'ngrams' para cada línea del corpus**

In [11]:
getNgramsRDD =twittsRDD.map(lambda x: ngrams(x, 2))
print(getNgramsRDD.collect())

[['Protester stage', 'stage diein', 'diein protest', 'protest store', 'store nyc', 'nyc anger', 'anger misplace', 'misplace retweet', 'retweet agree'], ['apple intraday', 'intraday comment', 'comment update', 'update range', 'range premium', 'premium user', 'user aapl', 'aapl stockaction', 'stockaction trading', 'trading stock', 'stock mkt'], ['ios app', 'app 2014', '2014 name', 'name elevate', 'elevate brain', 'brain training', 'training iphone', 'iphone app'], [], ['founder attack', 'attack boss', 'boss ridiculous', 'ridiculous comment', 'comment ad'], ['evidence factcheckthis'], [], ['hard reach', 'reach buy', 'buy suicide', 'suicide squad', 'squad comic_strip', 'comic_strip apple', 'apple store', 'store turkey'], ['delete music', 'music customer', 'customer ipod'], ['apple intraday', 'intraday comment', 'comment update', 'update range', 'range premium', 'premium user', 'user aapl', 'aapl stockaction', 'stockaction trading', 'trading stock', 'stock mkt'], ['studio 45,000', '45,000 o